### Testing on the looping procedure

In [11]:
import streamlit as st
import warnings
import locale
locale.setlocale(locale.LC_ALL, 'en_GB.UTF-8')
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import numpy_financial as npf
import requests
import math
from loan_amortization import loan_amortization, loan_amortization_custom_payment
from parameter import *

In [12]:

class Calculator():
    def __init__(self):
        
        # COMMENT THIS IF FLASK CONNECTION IS WORKING
        self.cpi = 0.12
        self.markup_percentage = 0.001
        self.maintenance_ratio = 0.08
        self.warranty_rate = 0.05
        self.insurance_rate = 0.015
        self.travel_labor_cost = 300
        self.business_con_rate = 0.02
        self.monthly_interest_rate = self.cpi/12
        # self.monthly_interest_rate = ((1+self.cpi)**(1/12))-1
        
        # UNCOMMENT THIS IF FLASK CONNECTION IS WORKING
        response = requests.get("http://127.0.0.1:5000/get_parameters")
        if response.status_code == 200:
            params = response.json()
            self.cpi = params['cpi']
            self.markup_percentage = params['markup_percentage']
            self.maintenance_ratio = params['maintenance_ratio']
            self.warranty_rate = params['warranty_rate']
            self.insurance_rate = params['insurance_rate']
            self.travel_labor_cost = params['travel_labor_cost']
            self.business_con_rate = params['business_con_rate']
        else:
            st.error("Failed to load parameters!")
            
        self.name = None
        self.monthlyPayment = None
        self.totalPayment = None
        self.invoice = None
                 


    def getMonthlyPayment(self, EquipmentPrice, LoanTerm, terminal_rate, warranty_yrs, insurance='Yes', maintenance='Yes', extra_warranty=0, bussiness_con='Yes'): 
        markup_price = EquipmentPrice # as it has been mark upped in the input form
        # maintenance_fee = (markup_price * self.maintenance_ratio if markup_price > 2500 else 0) if maintenance == 'Yes' else 0 
        maintenance_fee = (markup_price * self.maintenance_ratio) if maintenance == 'Yes' else 0 
 
        # warranty_yrs = 1 if markup_price <= 2000 else 2 if markup_price <= 5000 else 3 if markup_price <= 10000 else 5 if markup_price <= 30000 else 10
        additional_warranty = extra_warranty
        warranty_fee = markup_price * self.warranty_rate * (warranty_yrs + additional_warranty)
        insurance_fee = markup_price * self.insurance_rate * (warranty_yrs + additional_warranty) if insurance == 'Yes' else 0
        travel_labor_cost = self.travel_labor_cost * LoanTerm
        business_con_fee = (markup_price * self.business_con_rate * (warranty_yrs + additional_warranty) if insurance == 'Yes' else 0) if bussiness_con == 'Yes' else 0
        terminal_value = markup_price * terminal_rate * (warranty_yrs + additional_warranty)

        total_before_travel_labor = markup_price + maintenance_fee + warranty_fee + insurance_fee + business_con_fee + terminal_value
        total_payment = total_before_travel_labor + travel_labor_cost
        
        monthly_interest_rate = self.monthly_interest_rate

        monthlyPayment = npf.pmt(monthly_interest_rate, LoanTerm*12, -total_payment)

        self.monthlyPayment = monthlyPayment
        
        results = {
            "total_before_travel_labor": total_before_travel_labor,
            "total_payment": total_payment,
            "monthlyPayment": monthlyPayment,
            "maintenance_fee": maintenance_fee,
            "warranty_fee": warranty_fee,
            "insurance_fee": insurance_fee,
            "business_con_fee": business_con_fee,
            "terminal_value_fee": terminal_value
        }

        return results
    
    def getLoanTerm(self, EquipmentPrice, monthlyPayment, terminal_rate, insurance='Yes', maintenance='Yes', extra_warranty=0, bussiness_con='Yes'): 
        # Calculating the markup price
        markup_price = EquipmentPrice # as it has been mark upped in the input form
        
        # Calculating the maintenance fee
        maintenance_fee = (markup_price * self.maintenance_ratio if markup_price > 2500 else 0) if maintenance == 'Yes' else 0 
        
        # Determining the warranty years based on markup price
        warranty_yrs = 1 if markup_price <= 2000 else 2 if markup_price <= 5000 else 3 if markup_price <= 10000 else 5 if markup_price <= 30000 else 10
        
        # Calculating the warranty fee
        additional_warranty = extra_warranty
        warranty_fee = markup_price * self.warranty_rate * (warranty_yrs + additional_warranty)
        
        # Calculating the insurance fee
        insurance_fee = markup_price * self.insurance_rate * (warranty_yrs + additional_warranty) if insurance == 'Yes' else 0
        
        # Travel labor cost
        travel_labor_cost = self.travel_labor_cost
        
        # Calculating the terminal value
        terminal_value = markup_price * terminal_rate
        
        # Calculating the business continuity fee
        business_con_fee = markup_price * self.business_con_rate * (warranty_yrs + additional_warranty) if bussiness_con == 'Yes' else 0
        
        # Total before adding terminal value
        total_before_terminal = markup_price + maintenance_fee + warranty_fee + insurance_fee + business_con_fee
        
        # Total payment including terminal value and travel labor cost
        total_payment = total_before_terminal + terminal_value + travel_labor_cost
        
        # Define monthly interest rate
        monthly_interest_rate = self.monthly_interest_rate
        print(monthly_interest_rate)
        
        # Calculate the LoanTerm in months (can be decimal)
        LoanTerm_months = npf.nper(monthly_interest_rate, -monthlyPayment, total_payment, 1)
        print(LoanTerm_months)
        
        # total_payment_pv = npf.pv(monthly_interest_rate, LoanTerm_months, -monthlyPayment, -1)
        
        # Round up to the nearest whole number of months
        LoanTerm_months_rounded = math.ceil(LoanTerm_months)
        
        # Step 3: Calculate the remaining balance after making nper_rounded - 1 payments
        remaining_balance = npf.fv(monthly_interest_rate, LoanTerm_months_rounded - 1, -monthlyPayment, total_payment)

        # Step 4: Calculate the final payment to settle the remaining balance
        final_payment = remaining_balance * (1 + monthly_interest_rate)
        
        last_monthlyPayment = -final_payment
        
        # Calculate the LoanTerm in years
        LoanTerm_years = LoanTerm_months_rounded / 12

        # Save the loan term and the last month's payment
        self.LoanTerm = LoanTerm_years
        self.last_monthlyPayment = last_monthlyPayment

        results = {
            "total_before_terminal": total_before_terminal,
            "total_payment": total_payment,
            "LoanTerm_years": LoanTerm_years,
            "LoanTerm_months": LoanTerm_months_rounded,
            "last_monthlyPayment": last_monthlyPayment,
            "maintenance_fee": maintenance_fee,
            "warranty_fee": warranty_fee,
            "insurance_fee": insurance_fee,
            "terminal_value_fee": terminal_value,
            "business_con_fee": business_con_fee,
        }

        return results


    def getInvoice(self, EquipmentPrice, LoanTerm, terminal_rate, insurance, maintenance, extra_warranty, bussiness_con ): 
        invoice = self.getMonthlyPayment(EquipmentPrice, LoanTerm, terminal_rate, insurance, maintenance, extra_warranty, bussiness_con)['monthlyPayment'] * LoanTerm * 12
        return invoice

    def setName(self, name):
        self.name = name

    def displayResult(self, result):
        st.subheader(f"{self.name}")
        st.write(f"{result}")

In [13]:
import pandas as pd
from parameter import *

# Initialize an empty list to store the data
data = []

# Set initial value for EquipmentPrice
a = 5000

# Loop to generate data for 200 entries
for i in range(100):
    equipment_price = a
    loan_term = get_pricing_details(equipment_price)['Free Warranty Years']
    data.append({'EquipmentPrice': equipment_price, 'LoanTerm': loan_term})
    a += 2500

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(data)


In [14]:
clinic = Calculator()

list = []

for i in range(100):

    results = clinic.getMonthlyPayment(df['EquipmentPrice'][i], df['LoanTerm'][i],df['LoanTerm'][i],0)
    list.append(results['monthlyPayment'])
    
    

    

In [15]:
df['EquipmentPrice'].values

array([  5000,   7500,  10000,  12500,  15000,  17500,  20000,  22500,
        25000,  27500,  30000,  32500,  35000,  37500,  40000,  42500,
        45000,  47500,  50000,  52500,  55000,  57500,  60000,  62500,
        65000,  67500,  70000,  72500,  75000,  77500,  80000,  82500,
        85000,  87500,  90000,  92500,  95000,  97500, 100000, 102500,
       105000, 107500, 110000, 112500, 115000, 117500, 120000, 122500,
       125000, 127500, 130000, 132500, 135000, 137500, 140000, 142500,
       145000, 147500, 150000, 152500, 155000, 157500, 160000, 162500,
       165000, 167500, 170000, 172500, 175000, 177500, 180000, 182500,
       185000, 187500, 190000, 192500, 195000, 197500, 200000, 202500,
       205000, 207500, 210000, 212500, 215000, 217500, 220000, 222500,
       225000, 227500, 230000, 232500, 235000, 237500, 240000, 242500,
       245000, 247500, 250000, 252500], dtype=int64)

In [16]:
list

[209.25015182096234,
 298.9287883156605,
 388.6074248103586,
 478.2860613050568,
 567.964697799755,
 657.6433342944531,
 747.3219707891512,
 573.9067502704655,
 633.9667590197002,
 694.026767768935,
 754.0867765181698,
 814.1467852674045,
 874.2067940166393,
 934.266802765874,
 994.3268115151088,
 1054.3868202643437,
 1114.4468290135783,
 1174.5068377628131,
 1234.566846512048,
 1037.9806884679401,
 1085.6430670200393,
 1133.3054455721388,
 1180.967824124238,
 1228.6302026763374,
 1276.2925812284366,
 1323.9549597805358,
 1371.6173383326352,
 1419.2797168847344,
 1466.9420954368338,
 1514.604473988933,
 1562.2668525410324,
 1609.9292310931316,
 1657.591609645231,
 1705.2539881973303,
 1752.9163667494295,
 1800.5787453015287,
 1848.2411238536279,
 1895.9035024057273,
 1943.5658809578265,
 1631.2646833374179,
 1670.0018394061165,
 1708.738995474815,
 1747.4761515435134,
 1786.213307612212,
 1824.9504636809106,
 1863.6876197496092,
 1902.4247758183078,
 1941.1619318870062,
 1979.899087955

In [17]:
import seaborn as sns
import matplotlib.pyplot as plt

df['payment'] = list

# create sns line plot of list 
plt.figure(figsize=(10, 6))
sns.lineplot(data = df, x = 'EquipmentPrice', y='payment')
plt.show()

In [18]:
import seaborn as sns
import matplotlib.pyplot as plt

df['payment'] = list

# create sns line plot of list 
plt.figure(figsize=(10, 6))
sns.lineplot(data = df, x = 'EquipmentPrice', y='payment')

plt.axvline(x=df['EquipmentPrice'].iloc[3], color='red', linestyle='--', label='Loan Term = 2 yrs')
plt.axvline(x=df['EquipmentPrice'].iloc[9], color='red', linestyle='--', label='Loan Term = 2 yrs')
plt.axvline(x=df['EquipmentPrice'].iloc[19], color='red', linestyle='--', label='Loan Term = 2 yrs')

plt.xlabel('Equipment Price')
plt.ylabel('Monthly Payment')
# plt.legend()
plt.savefig('test.png')

### Generate Random DF

In [19]:
import random

In [20]:
import pandas as pd

# Define the data as a dictionary with numerical representations
data = {
    'Class': [1, 2, 3, 4, 5],
    'Bottom Price': [0, 20000, 50000, 10000, 200000],
    'Upper Price': [20000, 50000, 100000, 200000, 1000000],
    'Free Warranty Years': [3, 5, 7, 10, 10],
    'Markup Price': [5.0, 4.0, 2.0, 1.0, 1.0],
    'Warranty': [10.0, 5.0, 3.0, 1.0, 1.0],
    'Maintenance': [15.0, 10.0, 8.0, 5.0, 5.0],
    'Insurance': [10.0, 7.0, 5.0, 1.5, 1.5],
    'Business Con': [10.0, 5.0, 3.0, 2.0, 2.0],
    'Terminal Value': [5.0, 5.0, 3.0, 1.0, 1.0]
}

# Create a DataFrame
df_numeric = pd.DataFrame(data)



In [21]:
# Define the function to generate random data
def generate_random_data_custom_rules(df, max_values):
    """
    Generate random integer data for the DataFrame based on specified rules.
    All columns except 'Free Warranty Years' should have values in non-increasing order.
    'Free Warranty Years' should have values in non-decreasing order.

    Parameters:
        df (pd.DataFrame): The original DataFrame containing the data.
        max_values (dict): A dictionary specifying the maximum possible value for each column.
    
    Returns:
        pd.DataFrame: DataFrame with randomly generated integer values ordered based on rules.
    """
    
    # Create a copy of the dataframe to avoid modifying the original
    df_random = df.copy()

    # Iterate through each column to generate random values
    for column in max_values:
        # Generate five random values freely within the given range
        random_values = [random.randint(1, max_values[column]) for _ in range(5)]
        
        if column == 'Free Warranty Years':
            # Sort 'Free Warranty Years' in non-decreasing order (ascending)
            random_values.sort()
        else:
            # Sort other columns in non-increasing order (descending)
            random_values.sort(reverse=True)
        
        # Assign the sorted values back to the DataFrame
        df_random[column] = random_values

    return df_random






In [22]:
# Maximum possible values for each column
max_values = {
    'Free Warranty Years': 10,
    'Markup Price': 10,
    'Warranty': 10,
    'Maintenance': 15,
    'Insurance': 10,
    'Business Con': 10,
    'Terminal Value': 5
}


df_random = generate_random_data_custom_rules(df_numeric, max_values=max_values)

In [23]:
def get_pricing_details_from_df(equipment_price, df):
    """
    Function to get pricing details based on the equipment price from a given DataFrame.
    
    Parameters:
        equipment_price (float): The price of the equipment.
        df (pd.DataFrame): The DataFrame containing pricing data.
        
    Returns:
        dict: A dictionary containing the pricing details for the corresponding class.
    """
    
    # Find the row where the equipment price falls between the bottom and upper price
    row = df[(df['Bottom Price'] < equipment_price) & (df['Upper Price'] >= equipment_price)]
    
    # If no class matches, return None
    if row.empty:
        return None
    
    # Convert the row to a dictionary and return
    pricing_details = row.iloc[0].to_dict()
    return pricing_details

In [24]:
get_pricing_details_from_df(30000,df_random)

{'Class': 2,
 'Bottom Price': 20000,
 'Upper Price': 50000,
 'Free Warranty Years': 4,
 'Markup Price': 9,
 'Warranty': 4,
 'Maintenance': 10,
 'Insurance': 6,
 'Business Con': 7,
 'Terminal Value': 4}

### Running on random df generation

In [25]:
import pandas as pd
from parameter import *

# Initialize an empty list to store the data
data = []

# Set initial value for EquipmentPrice
a = 5000

# Loop to generate data for 200 entries
for i in range(100):
    equipment_price = a
    loan_term = get_pricing_details_from_df(equipment_price,df_random)['Free Warranty Years']
    data.append({'EquipmentPrice': equipment_price, 'LoanTerm': loan_term})
    a += 2500

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(data)


In [26]:
clinic = Calculator()

list = []

for i in range(100):
    
    

    results = clinic.getMonthlyPayment(df['EquipmentPrice'][i], df['LoanTerm'][i],df['LoanTerm'][i],0)
    list.append(results['monthlyPayment'])

In [27]:
import seaborn as sns
import matplotlib.pyplot as plt

df['payment'] = list

# create sns line plot of list 
plt.figure(figsize=(10, 6))
sns.lineplot(data = df, x = 'EquipmentPrice', y='payment')

plt.axvline(x=df['EquipmentPrice'].iloc[3], color='red', linestyle='--', label='Loan Term = 2 yrs')
plt.axvline(x=df['EquipmentPrice'].iloc[9], color='red', linestyle='--', label='Loan Term = 2 yrs')
plt.axvline(x=df['EquipmentPrice'].iloc[19], color='red', linestyle='--', label='Loan Term = 2 yrs')

plt.xlabel('Equipment Price')
plt.ylabel('Monthly Payment')
# plt.legend()
plt.savefig('test.png')

### Build as function

In [28]:
import pandas as pd
from parameter import *
import seaborn as sns
import matplotlib.pyplot as plt
from parameter import *

In [29]:
import streamlit as st
import warnings
import locale
locale.setlocale(locale.LC_ALL, 'en_GB.UTF-8')
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import numpy_financial as npf
import requests
import math
from loan_amortization import loan_amortization, loan_amortization_custom_payment
from parameter import *



class Calculator():
    def __init__(self):
        
        # COMMENT THIS IF FLASK CONNECTION IS WORKING
        self.cpi = 0.12
        self.markup_percentage = 0.001
        self.maintenance_ratio = 0.08
        self.warranty_rate = 0.05
        self.insurance_rate = 0.015
        self.travel_labor_cost = 300
        self.business_con_rate = 0.02
        self.monthly_interest_rate = self.cpi/12
        # self.monthly_interest_rate = ((1+self.cpi)**(1/12))-1
        
        # UNCOMMENT THIS IF FLASK CONNECTION IS WORKING
        response = requests.get("http://127.0.0.1:5000/get_parameters")
        if response.status_code == 200:
            params = response.json()
            self.cpi = params['cpi']
            self.markup_percentage = params['markup_percentage']
            self.maintenance_ratio = params['maintenance_ratio']
            self.warranty_rate = params['warranty_rate']
            self.insurance_rate = params['insurance_rate']
            self.travel_labor_cost = params['travel_labor_cost']
            self.business_con_rate = params['business_con_rate']
        else:
            st.error("Failed to load parameters!")
            
        self.name = None
        self.monthlyPayment = None
        self.totalPayment = None
        self.invoice = None
                 


    def getMonthlyPayment(self, EquipmentPrice, LoanTerm, terminal_rate, warranty_yrs, insurance='Yes', maintenance='Yes', extra_warranty=0, bussiness_con='Yes'): 
        markup_price = EquipmentPrice # as it has been mark upped in the input form
        # maintenance_fee = (markup_price * self.maintenance_ratio if markup_price > 2500 else 0) if maintenance == 'Yes' else 0 
        maintenance_fee = (markup_price * self.maintenance_ratio) if maintenance == 'Yes' else 0 
 
        # warranty_yrs = 1 if markup_price <= 2000 else 2 if markup_price <= 5000 else 3 if markup_price <= 10000 else 5 if markup_price <= 30000 else 10
        additional_warranty = extra_warranty
        warranty_fee = markup_price * self.warranty_rate * (warranty_yrs + additional_warranty)
        insurance_fee = markup_price * self.insurance_rate * (warranty_yrs + additional_warranty) if insurance == 'Yes' else 0
        travel_labor_cost = self.travel_labor_cost * LoanTerm
        business_con_fee = (markup_price * self.business_con_rate * (warranty_yrs + additional_warranty) if insurance == 'Yes' else 0) if bussiness_con == 'Yes' else 0
        terminal_value = markup_price * terminal_rate * (warranty_yrs + additional_warranty)

        total_before_travel_labor = markup_price + maintenance_fee + warranty_fee + insurance_fee + business_con_fee + terminal_value
        total_payment = total_before_travel_labor + travel_labor_cost
        
        monthly_interest_rate = self.monthly_interest_rate

        monthlyPayment = npf.pmt(monthly_interest_rate, LoanTerm*12, -total_payment)

        self.monthlyPayment = monthlyPayment
        
        results = {
            "total_before_travel_labor": total_before_travel_labor,
            "total_payment": total_payment,
            "monthlyPayment": monthlyPayment,
            "maintenance_fee": maintenance_fee,
            "warranty_fee": warranty_fee,
            "insurance_fee": insurance_fee,
            "business_con_fee": business_con_fee,
            "terminal_value_fee": terminal_value,
            'travel_labor_cost': travel_labor_cost
        }

        return results
    
    def getLoanTerm(self, EquipmentPrice, monthlyPayment, terminal_rate, insurance='Yes', maintenance='Yes', extra_warranty=0, bussiness_con='Yes'): 
        # Calculating the markup price
        markup_price = EquipmentPrice # as it has been mark upped in the input form
        
        # Calculating the maintenance fee
        maintenance_fee = (markup_price * self.maintenance_ratio if markup_price > 2500 else 0) if maintenance == 'Yes' else 0 
        
        # Determining the warranty years based on markup price
        warranty_yrs = 1 if markup_price <= 2000 else 2 if markup_price <= 5000 else 3 if markup_price <= 10000 else 5 if markup_price <= 30000 else 10
        
        # Calculating the warranty fee
        additional_warranty = extra_warranty
        warranty_fee = markup_price * self.warranty_rate * (warranty_yrs + additional_warranty)
        
        # Calculating the insurance fee
        insurance_fee = markup_price * self.insurance_rate * (warranty_yrs + additional_warranty) if insurance == 'Yes' else 0
        
        # Travel labor cost
        travel_labor_cost = self.travel_labor_cost
        
        # Calculating the terminal value
        terminal_value = markup_price * terminal_rate
        
        # Calculating the business continuity fee
        business_con_fee = markup_price * self.business_con_rate * (warranty_yrs + additional_warranty) if bussiness_con == 'Yes' else 0
        
        # Total before adding terminal value
        total_before_terminal = markup_price + maintenance_fee + warranty_fee + insurance_fee + business_con_fee
        
        # Total payment including terminal value and travel labor cost
        total_payment = total_before_terminal + terminal_value + travel_labor_cost
        
        # Define monthly interest rate
        monthly_interest_rate = self.monthly_interest_rate
        print(monthly_interest_rate)
        
        # Calculate the LoanTerm in months (can be decimal)
        LoanTerm_months = npf.nper(monthly_interest_rate, -monthlyPayment, total_payment, 1)
        print(LoanTerm_months)
        
        # total_payment_pv = npf.pv(monthly_interest_rate, LoanTerm_months, -monthlyPayment, -1)
        
        # Round up to the nearest whole number of months
        LoanTerm_months_rounded = math.ceil(LoanTerm_months)
        
        # Step 3: Calculate the remaining balance after making nper_rounded - 1 payments
        remaining_balance = npf.fv(monthly_interest_rate, LoanTerm_months_rounded - 1, -monthlyPayment, total_payment)

        # Step 4: Calculate the final payment to settle the remaining balance
        final_payment = remaining_balance * (1 + monthly_interest_rate)
        
        last_monthlyPayment = -final_payment
        
        # Calculate the LoanTerm in years
        LoanTerm_years = LoanTerm_months_rounded / 12

        # Save the loan term and the last month's payment
        self.LoanTerm = LoanTerm_years
        self.last_monthlyPayment = last_monthlyPayment

        results = {
            "total_before_terminal": total_before_terminal,
            "total_payment": total_payment,
            "LoanTerm_years": LoanTerm_years,
            "LoanTerm_months": LoanTerm_months_rounded,
            "last_monthlyPayment": last_monthlyPayment,
            "maintenance_fee": maintenance_fee,
            "warranty_fee": warranty_fee,
            "insurance_fee": insurance_fee,
            "terminal_value_fee": terminal_value,
            "business_con_fee": business_con_fee,
        }

        return results


    def getInvoice(self, EquipmentPrice, LoanTerm, terminal_rate, insurance, maintenance, extra_warranty, bussiness_con ): 
        invoice = self.getMonthlyPayment(EquipmentPrice, LoanTerm, terminal_rate, insurance, maintenance, extra_warranty, bussiness_con)['monthlyPayment'] * LoanTerm * 12
        return invoice

    def setName(self, name):
        self.name = name

    def displayResult(self, result):
        st.subheader(f"{self.name}")
        st.write(f"{result}")

In [30]:
# Maximum possible values for each column
max_values = {
    'Free Warranty Years': 10,
    'Markup Price': 10,
    'Warranty': 10,
    'Maintenance': 15,
    'Insurance': 10,
    'Business Con': 10,
    'Terminal Value': 5
}


In [43]:
def run_random_sim():

    # randomize parameter configuration
    df_random = generate_random_data_custom_rules(df_numeric, max_values=max_values)

    # Initialize an empty list to store the data
    data = []
    params = []

    # Set initial value for EquipmentPrice
    a = 5000

    # Loop to generate data for 200 entries
    for i in range(120):
        equipment_price = a
        response = get_pricing_details_from_df(equipment_price,df_random)
        loan_term = response['Free Warranty Years']
        data.append({'EquipmentPrice': equipment_price, 'LoanTerm': loan_term})
        
        params.append({
            'Markup Price': response['Markup Price'],
            'Warranty': response['Warranty'],
            'Maintenance': response['Maintenance'],
            'Insurance': response['Insurance'],
            'Business Con': response['Business Con'],
            'Terminal Value': response['Terminal Value']
            
        })
        
        a += 2500

    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(data)
    
    clinic = Calculator()

    list = []
    total_payment = []
    finance_product = []

    for i in range(120):
        
        clinic.markup_percentage = params[i]['Markup Price'] / 100
        clinic.warranty_rate = params[i]['Warranty'] / 100
        clinic.maintenance_ratio = params[i]['Maintenance'] / 100
        clinic.insurance_rate = params[i]['Insurance'] / 100
        clinic.business_con_rate = params[i]['Business Con'] / 100
        warranty = df['LoanTerm'][i] - 2 if df['LoanTerm'][i] - 2 > 0 else 1
        terminal_rate = params[i]['Terminal Value'] / 100
        
        
        results = clinic.getMonthlyPayment(df['EquipmentPrice'][i], df['LoanTerm'][i],warranty,terminal_rate)
        list.append(results['monthlyPayment'])
        
        
        
        total_payment.append(results['monthlyPayment'] * (df['LoanTerm'][i]*12))
        
        
        
        finance_product_profit = results['maintenance_fee'] + results['warranty_fee'] + results['insurance_fee'] + results['business_con_fee'] + results['terminal_value_fee'] + results['travel_labor_cost']
        finance_product.append(finance_product_profit)
        
        
    df['payment'] = list
    df['total_payment'] = total_payment
    df['finance_product_gain'] = finance_product
    df['total_interest'] = df['total_payment'] - df['EquipmentPrice'] - df['finance_product_gain']
    df['total_gain']  = df['total_payment'] - df['EquipmentPrice']

    # create sns line plot of list 
    plt.figure(figsize=(10, 6))
    sns.lineplot(data = df, x = 'EquipmentPrice', y='payment')

    plt.axvline(x=df['EquipmentPrice'].iloc[6], color='red', linestyle='--', label='Loan Term = 2 yrs')
    plt.axvline(x=df['EquipmentPrice'].iloc[18], color='red', linestyle='--', label='Loan Term = 2 yrs')
    plt.axvline(x=df['EquipmentPrice'].iloc[38], color='red', linestyle='--', label='Loan Term = 2 yrs')
    plt.axvline(x=df['EquipmentPrice'].iloc[78], color='red', linestyle='--', label='Loan Term = 2 yrs')

    plt.xlabel('Equipment Price')
    plt.ylabel('Monthly Payment')
    # plt.legend()
    # plt.show()
    # plt.savefig('test.png')
    
    return df_random, df['total_gain'].mean(), df['payment'].max()
        


In [64]:
df, avg_gain, max_payment = run_random_sim()
df

,Class,Bottom Price,Upper Price,Free Warranty Years,Markup Price,Warranty,Maintenance,Insurance,Business Con,Terminal Value
0,1,0,20000,1,10,9,12,9,10,5
1,2,20000,50000,2,6,8,10,6,10,4
2,3,50000,100000,5,6,5,8,5,9,3
3,4,10000,200000,6,5,2,7,3,8,2
4,5,200000,1000000,8,5,2,7,2,3,1


In [63]:
best_avg_gain = 0
best_df = None

upper_bound = 2000

for i in range(5):
    df, avg_gain, max_payment = run_random_sim()
    print(max_payment)
    
    if avg_gain > best_avg_gain :
        if max_payment <= upper_bound:
            best_avg_gain = avg_gain
            best_df = df
        
    

719701.8429311711
275457.8995394627
241695.35984169287
198323.67221343354
250580.23870952707


In [60]:
best_avg_gain

0

In [48]:
# rename column 'Free Warranty Years' to 'Loan Term'
best_df.rename(columns={'Free Warranty Years': 'Loan Term'}, inplace=True)
best_df

AttributeError: 'NoneType' object has no attribute 'rename'

In [41]:
def run_best_sim():

    # randomize parameter configuration
    df_random = best_df

    # Initialize an empty list to store the data
    data = []
    params = []

    # Set initial value for EquipmentPrice
    a = 5000

    # Loop to generate data for 200 entries
    for i in range(120):
        equipment_price = a
        response = get_pricing_details_from_df(equipment_price,df_random)
        loan_term = response['Loan Term']
        data.append({'EquipmentPrice': equipment_price, 'LoanTerm': loan_term})
        
        params.append({
            'Markup Price': response['Markup Price'],
            'Warranty': response['Warranty'],
            'Maintenance': response['Maintenance'],
            'Insurance': response['Insurance'],
            'Business Con': response['Business Con'],
            'Terminal Value': response['Terminal Value']
            
        })
        
        a += 2500

    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(data)
    
    clinic = Calculator()

    list = []
    total_payment = []
    finance_product = []

    for i in range(120):
        
        clinic.markup_percentage = params[i]['Markup Price']
        clinic.warranty_rate = params[i]['Warranty']
        clinic.maintenance_ratio = params[i]['Maintenance']
        clinic.insurance_rate = params[i]['Insurance']
        clinic.business_con_rate = params[i]['Business Con']
        warranty = df['LoanTerm'][i] - 2 if df['LoanTerm'][i] - 2 > 0 else 1
        terminal_rate = params[i]['Terminal Value']
        
        
        results = clinic.getMonthlyPayment(df['EquipmentPrice'][i], df['LoanTerm'][i],warranty,terminal_rate)
        list.append(results['monthlyPayment'])
        
        
        
        total_payment.append(results['monthlyPayment'] * (df['LoanTerm'][i]*12))
        
        
        
        finance_product_profit = results['maintenance_fee'] + results['warranty_fee'] + results['insurance_fee'] + results['business_con_fee'] + results['terminal_value_fee'] + results['travel_labor_cost']
        finance_product.append(finance_product_profit)
        
        
    df['payment'] = list
    df['total_payment'] = total_payment
    df['finance_product_gain'] = finance_product
    df['total_interest'] = df['total_payment'] - df['EquipmentPrice'] - df['finance_product_gain']
    df['total_gain']  = df['total_payment'] - df['EquipmentPrice']

    # create sns line plot of list 
    plt.figure(figsize=(10, 6))
    sns.lineplot(data = df, x = 'EquipmentPrice', y='payment')

    plt.axvline(x=df['EquipmentPrice'].iloc[6], color='red', linestyle='--', label='Loan Term = 2 yrs')
    plt.axvline(x=df['EquipmentPrice'].iloc[18], color='red', linestyle='--', label='Loan Term = 2 yrs')
    plt.axvline(x=df['EquipmentPrice'].iloc[38], color='red', linestyle='--', label='Loan Term = 2 yrs')
    plt.axvline(x=df['EquipmentPrice'].iloc[78], color='red', linestyle='--', label='Loan Term = 2 yrs')

    plt.xlabel('Equipment Price')
    plt.ylabel('Monthly Payment')
    # plt.legend()
    plt.show()
    plt.savefig('test.png')
    

        


In [42]:
run_best_sim()